# Предварительный анализ и обработка данных

> Для корректной работы ссылок оглавления лучше смотреть проект здесь \
> https://nbviewer.org/github/experiment0/sf_data_science/blob/main/project_08/2_research_and_prepare_data.ipynb

## Оглавление

- [Загрузка данных](#load) 
- [Анализ распределения основных характеристик признаков](#research_dist) 
- [Удаление пропусков и дубликатов](#clear) 
- [Изучение видов транзакций](#research_invoice_no) 
- [Исследование транзакций с отрицательным количеством товаров](#research_neq_quantity)
- [Исследование транзакций с нулевой стоимостью товаров](#research_zero_price)
- [Добавление признака TotalPrice (общая стоимость покупки)](#total_price)

## Загрузка данных <a id="load"></a>

Загрузим необходимые библиотеки и вспомогательные функции.

In [1]:
import pandas as pd

from helpers.prepare_data import (
    F, FT,
    get_splited_data,
    get_str_prefix,
    get_str_postfix,
    get_quantity_canceled,
    get_total_price,
)

from warnings import simplefilter
simplefilter('ignore')

Загрузим данные.

In [2]:
source_data = pd.read_csv(
    './data/customer_segmentation_project.csv',
    encoding='ISO-8859-1', 
    dtype={F.CUSTOMER_ID.value: str, F.INVOICE_NO.value: str}
)

# выделим тренировочную часть с помощью подготовленной функции
data, sample_data = get_splited_data(source_data)

data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


**Описание столбцов:**

- `InvoiceNo` — номер счёта-фактуры \
(уникальный шестизначный номер, присваиваемый каждой транзакции; \
буква "C" в начале кода указывает на отмену транзакции);
- `StockCode` — код товара \
(уникальное пятизначное целое число, присваиваемое каждому отдельному товару);
- `Description` — название товара;
- `Quantity` — количество каждого товара за транзакцию;
- `InvoiceDate` — дата и время выставления счёта/проведения транзакции;
- `UnitPrice` — цена за единицу товара в фунтах стерлингов;
- `CustomerID` — идентификатор клиента \
(уникальный пятизначный номер, однозначно присваиваемый каждому клиенту);
- `Country` — название страны, в которой проживает клиент.

## Анализ распределения основных характеристик признаков <a id="research_dist"></a>

In [3]:
print('Размерность данных: ', data.shape)

Размерность данных:  (500380, 8)


Посмотрим на типы данных в столбцах.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500380 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    500380 non-null  object 
 1   StockCode    500380 non-null  object 
 2   Description  498926 non-null  object 
 3   Quantity     500380 non-null  int64  
 4   InvoiceDate  500380 non-null  object 
 5   UnitPrice    500380 non-null  float64
 6   CustomerID   365300 non-null  object 
 7   Country      500380 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 34.4+ MB


Столбец `InvoiceDate` далее переведем в тип `datetime`.\
Типы остальных столбцов соответствуют характеру данных, которые в них содержатся.

Проверим наличие пропусков.

In [5]:
data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Есть пропуски в столбцах с описанием товара и идентификатором клиента.\
Позже разберемся с ними.

Посмотрим, за какой период представлены данные.

In [6]:
data[F.INVOICE_DATE.value] = pd.to_datetime(data[F.INVOICE_DATE.value])

print('Начальная дата: ', data[F.INVOICE_DATE.value].min())
print('Конечная дата: ', data[F.INVOICE_DATE.value].max())

Начальная дата:  2010-12-01 08:26:00
Конечная дата:  2011-12-09 12:50:00


Далее изучим основные описательные характеристики числовых признаков транзакций.

In [7]:
data.describe()

,Quantity,UnitPrice
count,500380.000000,500380.000000
mean,9.442338,4.608206
std,171.289884,97.823992
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.100000
75%,10.000000,4.130000
max,80995.000000,38970.000000


Как видим, в транзакциях есть отрицательное количество товаров и отрицательная цена.\
Видимо, это возвраты ранее купленных товаров.\
Учтем это при обработке данных.

Посмотрим, есть ли нулевые значения этих признаков.

In [8]:
mask = data[F.QUANTITY.value] == 0

print('Количество транзакций с нулевым количеством товаров: ', data[mask].shape[0])

Количество транзакций с нулевым количеством товаров:  0


In [9]:
mask = data[F.UNIT_PRICE.value] == 0

print('Количество транзакций с нулевой стоимостью товаров: ', data[mask].shape[0])

Количество транзакций с нулевой стоимостью товаров:  2513


Посмотрим на характеристики этих же признаков с положительным значением.

In [10]:
mask = (data[F.QUANTITY.value] > 0) & (data[F.UNIT_PRICE.value] > 0)

data[mask].describe()

,Quantity,UnitPrice
count,489610.000000,489610.000000
mean,10.308374,3.926352
std,121.658445,34.238457
min,1.000000,0.001000
25%,1.000000,1.250000
50%,3.000000,2.100000
75%,10.000000,4.130000
max,80995.000000,13541.330000


Посмотрим на характеристики распределения признаков с типом `object`.

In [11]:
data.describe(include='object')

,InvoiceNo,StockCode,Description,CustomerID,Country
count,500380,500380,498926,365300,500380
unique,23553,4060,4206,3935,36
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,17841,United Kingdom
freq,1114,2125,2181,7983,456980


Итого, у нас

In [12]:
print('Количество уникальных товаров:', data[F.STOCK_CODE.value].nunique())
print('Количество уникальных покупателей:', data[F.CUSTOMER_ID.value].nunique())

Количество уникальных товаров: 4060
Количество уникальных покупателей: 3935


In [13]:
print('Идентификатор самого популярного товара:', data[F.STOCK_CODE.value].mode()[0])

Идентификатор самого популярного товара: 85123A


Посмотрим описания этого популярного товара.

In [14]:
mask = data[F.STOCK_CODE.value] == '85123A'
data[mask][F.DESCRIPTION.value].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    2114
CREAM HANGING HEART T-LIGHT HOLDER       9
?                                        1
wrongly marked carton 22804              1
Name: Description, dtype: int64

В переводе "Подвесной держатель для свечи в форме сердца кремового цвета".

Посмотрим на названия стран.

In [15]:
data[F.COUNTRY.value].unique()

array(['United Kingdom', 'France', 'Australia', 'Germany', 'Norway',
       'EIRE', 'Poland', 'Portugal', 'Italy', 'Belgium', 'Lithuania',
       'Japan', 'Iceland', 'Channel Islands', 'Spain', 'Cyprus', 'Sweden',
       'Austria', 'Israel', 'Finland', 'Switzerland', 'Netherlands',
       'Bahrain', 'Greece', 'Hong Kong', 'United Arab Emirates',
       'Denmark', 'Saudi Arabia', 'Czech Republic', 'Unspecified',
       'Brazil', 'USA', 'European Community', 'Canada', 'Malta', 'RSA'],
      dtype=object)

Есть страна с названием `Unspecified`, что можно расценивать как пропуск в данных.\
Посчитаем количество стран.

In [16]:
mask = data[F.COUNTRY.value] != 'Unspecified'

print('Количество уникальных стран:', data[mask][F.COUNTRY.value].nunique())

Количество уникальных стран: 35


## Удаление пропусков и дубликатов <a id="clear"></a>

Посмотрим на строки с пропуском в столбце `CustomerID`.

In [17]:
mask = data[F.CUSTOMER_ID.value].isna()

data[mask].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom


In [18]:
print('Количество строк:', data[mask].shape[0])

Количество строк: 135080


Посмотрим на распределение характеристик в этих строках с признаком типа `object`.

In [19]:
data[mask].describe(include='object')

,InvoiceNo,StockCode,Description,CustomerID,Country
count,135080,135080,133626,0,135080
unique,3710,3810,3554,0,9
top,573585,DOT,DOTCOM POSTAGE,NaN,United Kingdom
freq,1114,694,693,NaN,133600


Большинство транзакций здесь - это почтовые расходы (DOTCOM POSTAGE).

Посмотрим на пропуски.

In [20]:
data[mask].isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Видим, что все пропуски `Description` (1454 из исходных данных) попадают в данную выборку.

In [21]:
percent_of_absences = round(data[mask].shape[0] * 100 / data.shape[0], 2)

print(f'Строки с пропуском значения CustomerID составляют {percent_of_absences}% от общего количества данных.')

Строки с пропуском значения CustomerID составляют 27.0% от общего количества данных.


Пропуски в столбце `CustomerID` говорят о незавершенных или некорректных транзакциях.\
Удалим их.

In [22]:
data.dropna(subset=[F.CUSTOMER_ID.value], inplace=True)

In [23]:
print('Оставшееся количество пропусков в данных:', data.isna().sum().sum())

Оставшееся количество пропусков в данных: 0


Теперь посмотрим, сколько в данных дубликатов.

In [24]:
duplicated_mask = data.duplicated()
duplicated_count = data[duplicated_mask].shape[0]

print('Количество дубликатов: ', duplicated_count)

Количество дубликатов:  4722


Удалим дубликаты.

In [25]:
data.drop_duplicates(ignore_index=True, inplace=True)

## Изучение видов транзакций <a id="research_invoice_no"></a>

Посмотрим, с каким видом значений `InvoiceNo` и `StockCode` мы можем встретиться.\
Посмотрим, какие у них могут быть буквенные префиксы и окончания.

In [26]:
# Определим префикс и постфикс у номера счета-фактуры (InvoiceNo)
data[FT.INVOICE_NO_PREFIX.value] = data[F.INVOICE_NO.value].apply(get_str_prefix)
data[FT.INVOICE_NO_POSTFIX.value] = data[F.INVOICE_NO.value].apply(get_str_postfix)

print('Уникальные префиксы InvoiceNo: ')
print(data[FT.INVOICE_NO_PREFIX.value].unique())
print()
print('Уникальные окончания InvoiceNo: ')
print(data[FT.INVOICE_NO_POSTFIX.value].unique())

Уникальные префиксы InvoiceNo: 
['' 'C']

Уникальные окончания InvoiceNo: 
['']


То есть, для счетов-фактур у нас может быть только префикс `C`,\
который обозначает отмену транзации.

In [27]:
mask = data[FT.INVOICE_NO_PREFIX.value] == 'C'
print('Количество транзакций, которые являются отменами: ', data[mask].shape[0])

Количество транзакций, которые являются отменами:  7842


In [28]:
# Определим префикс и постфикс у кода товара (StockCode)
data[FT.STOCK_CODE_PREFIX.value] = data[F.STOCK_CODE.value].apply(get_str_prefix)
data[FT.STOCK_CODE_POSTFIX.value] = data[F.STOCK_CODE.value].apply(get_str_postfix)

print('Уникальные префиксы StockCode: ')
print(data[FT.STOCK_CODE_PREFIX.value].unique())
print()
print('Уникальные окончания StockCode: ')
print(data[FT.STOCK_CODE_POSTFIX.value].unique())

Уникальные префиксы StockCode: 
['' 'POST' 'D' 'C' 'M' 'BANK CHARGES' 'PADS' 'DOT' 'CRUK']

Уникальные окончания StockCode: 
['A' '' 'B' 'G' 'E' 'POST' 'L' 'C' 'S' 'BL' 'N' 'D' 'F' 'T' 'H' 'M' 'R'
 'K' 'P' 'W' 'BANK CHARGES' 'J' 'U' 'V' 'PADS' 'I' 'DOT' 'CRUK' 'Y']


Некоторые префиксы и окончания совпадают.\
Скорее всего `StockCode` в этих случаях состоит полностью из букв (соответствующих префиксу).\
Выделим такие случаи.

In [29]:
mask = (data[FT.STOCK_CODE_PREFIX.value] != '') & \
       (data[FT.STOCK_CODE_PREFIX.value] == data[FT.STOCK_CODE_POSTFIX.value])

data[mask][FT.STOCK_CODE_PREFIX.value].value_counts()

POST            1095
M                389
D                 70
DOT               16
CRUK              16
BANK CHARGES       8
PADS               4
Name: StockCodePrefix, dtype: int64

Посмотрим сами эти транзакции и их описания.

### Почтовые расходы (StockCode == POST)

In [30]:
mask = data[FT.STOCK_CODE_PREFIX.value] == 'POST'
data[mask].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNoPrefix,InvoiceNoPostfix,StockCodePrefix,StockCodePostfix
45,536370,POST,POSTAGE,3,2010-12-01 08:45:00,18.0,12583,France,,,POST,POST
1012,536527,POST,POSTAGE,1,2010-12-01 13:04:00,18.0,12662,Germany,,,POST,POST
3532,536840,POST,POSTAGE,1,2010-12-02 18:27:00,18.0,12738,Germany,,,POST,POST
3695,536852,POST,POSTAGE,1,2010-12-03 09:51:00,18.0,12686,France,,,POST,POST
3800,536861,POST,POSTAGE,3,2010-12-03 10:44:00,18.0,12427,Germany,,,POST,POST


Посмотрим на их описания.

In [31]:
data[mask][F.DESCRIPTION.value].value_counts()

POSTAGE    1095
Name: Description, dtype: int64

Все описания имеют значение "POSTAGE" - "ПОЧТОВЫЕ РАСХОДЫ".\
Пожалуй, подобные транзакции не характеризуют клиентов (разве что в том плане, что они живут в другой стране и им нужно доставлять товар).\
Поэтому удалим эти записи из таблицы.

In [32]:
data.drop(data[mask].index, inplace=True)

### Руководство к товару (StockCode == M)

In [33]:
mask = data[FT.STOCK_CODE_PREFIX.value] == 'M'
data[mask].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNoPrefix,InvoiceNoPostfix,StockCodePrefix,StockCodePostfix
1523,536569,M,Manual,1,2010-12-01 15:35:00,1.25,16274,United Kingdom,,,M,M
1534,536569,M,Manual,1,2010-12-01 15:35:00,18.95,16274,United Kingdom,,,M,M
4322,536981,M,Manual,2,2010-12-03 14:26:00,0.85,14723,United Kingdom,,,M,M
5149,537077,M,Manual,12,2010-12-05 11:59:00,0.42,17062,United Kingdom,,,M,M
5692,537137,M,Manual,36,2010-12-05 12:43:00,0.85,16327,United Kingdom,,,M,M


Посмотрим на описания.

In [34]:
data[mask][F.DESCRIPTION.value].value_counts()

Manual    389
Name: Description, dtype: int64

Все описания имеют значение "Manual" - "Руководство".\
Более подробное исследование данного значения приводит к мысли о том, что это руководство к товарам.\
Удалим эти значения, потому как самостоятельным товаром они не являются.

In [35]:
data.drop(data[mask].index, inplace=True)

### Скидки (StockCode == D)

In [36]:
mask = data[FT.STOCK_CODE_PREFIX.value] == 'D'
data[mask].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNoPrefix,InvoiceNoPostfix,StockCodePrefix,StockCodePostfix
139,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom,C,,D,D
6118,C537164,D,Discount,-1,2010-12-05 13:21:00,29.29,14527,United Kingdom,C,,D,D
9120,C537597,D,Discount,-1,2010-12-07 12:34:00,281.00,15498,United Kingdom,C,,D,D
11356,C537857,D,Discount,-1,2010-12-08 16:00:00,267.12,17340,United Kingdom,C,,D,D
18236,C538897,D,Discount,-1,2010-12-15 09:14:00,5.76,16422,United Kingdom,C,,D,D


Посмотрим на описания.

In [37]:
data[mask][F.DESCRIPTION.value].value_counts()

Discount    70
Name: Description, dtype: int64

Все транзакции имеют описание "Discount" - "Скидка".\
Посмотрим на распределение числовых признаков в таких транзакциях.

In [38]:
data[mask].describe()

,Quantity,UnitPrice
count,70.000000,70.000000
mean,-16.285714,74.806143
std,90.653048,229.291132
min,-720.000000,0.010000
25%,-1.000000,13.907500
50%,-1.000000,23.120000
75%,-1.000000,54.532500
max,-1.000000,1867.860000


`Quantity` (количество) в них всегда имеет отрицательное значение.\
Также удалим их, потому что они не являются самостоятельно покупаемыми товарами, \
а являются скидками на товары из заказа.

In [39]:
data.drop(data[mask].index, inplace=True)

### Почтовые расходы 2 (StockCode == DOT)

In [40]:
mask = data[FT.STOCK_CODE_PREFIX.value] == 'DOT'
data[mask].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNoPrefix,InvoiceNoPostfix,StockCodePrefix,StockCodePostfix
202403,564764,DOT,DOTCOM POSTAGE,1,2011-08-30 10:49:00,11.17,14096,United Kingdom,,,DOT,DOT
205367,565383,DOT,DOTCOM POSTAGE,1,2011-09-02 15:45:00,16.46,14096,United Kingdom,,,DOT,DOT
212531,566217,DOT,DOTCOM POSTAGE,1,2011-09-09 15:17:00,13.16,14096,United Kingdom,,,DOT,DOT
216613,566566,DOT,DOTCOM POSTAGE,1,2011-09-13 12:32:00,85.58,14096,United Kingdom,,,DOT,DOT
225445,567656,DOT,DOTCOM POSTAGE,1,2011-09-21 14:40:00,878.55,14096,United Kingdom,,,DOT,DOT


Посмотрим на описания.

In [41]:
data[mask][F.DESCRIPTION.value].value_counts()

DOTCOM POSTAGE    16
Name: Description, dtype: int64

Все описания имеют значение "DOTCOM POSTAGE" - "ДОТКОМ ПОЧТОВЫЕ РАСХОДЫ".\
Также удалим эти транзакции.

In [42]:
data.drop(data[mask].index, inplace=True)

### Комиссия (StockCode == CRUK)

In [43]:
mask = data[FT.STOCK_CODE_PREFIX.value] == 'CRUK'
data[mask].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNoPrefix,InvoiceNoPostfix,StockCodePrefix,StockCodePostfix
202404,C564763,CRUK,CRUK Commission,-1,2011-08-30 10:49:00,1.60,14096,United Kingdom,C,,CRUK,CRUK
205388,C565382,CRUK,CRUK Commission,-1,2011-09-02 15:45:00,13.01,14096,United Kingdom,C,,CRUK,CRUK
212555,C566216,CRUK,CRUK Commission,-1,2011-09-09 15:17:00,15.96,14096,United Kingdom,C,,CRUK,CRUK
216632,C566565,CRUK,CRUK Commission,-1,2011-09-13 12:32:00,52.24,14096,United Kingdom,C,,CRUK,CRUK
225849,C567655,CRUK,CRUK Commission,-1,2011-09-21 14:40:00,608.66,14096,United Kingdom,C,,CRUK,CRUK


Посмотрим на описания.

In [44]:
data[mask][F.DESCRIPTION.value].value_counts()

CRUK Commission    16
Name: Description, dtype: int64

Все описания имеют значение "CRUK Commission" - "Комиссия".\
Посмотрим на распределение числовых значений в данных транзакциях.

In [45]:
data[mask].describe()

,Quantity,UnitPrice
count,16.0,16.000000
mean,-1.0,495.839375
std,0.0,364.164786
min,-1.0,1.600000
25%,-1.0,284.252500
50%,-1.0,471.770000
75%,-1.0,668.977500
max,-1.0,1100.440000


Значение `Quantity` везде равно `-1`.\
Похоже, что это общая комиссия к каждому заказу.\
Также удалим эти транзакции.

In [46]:
data.drop(data[mask].index, inplace=True)

### Банковские сборы (StockCode == BANK CHARGES)

In [47]:
mask = data[FT.STOCK_CODE_PREFIX.value] == 'BANK CHARGES'
data[mask]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNoPrefix,InvoiceNoPostfix,StockCodePrefix,StockCodePostfix
2931,536779,BANK CHARGES,Bank Charges,1,2010-12-02 15:08:00,15.000,15823,United Kingdom,,,BANK CHARGES,BANK CHARGES
33701,541505,BANK CHARGES,Bank Charges,1,2011-01-18 15:58:00,15.000,15939,United Kingdom,,,BANK CHARGES,BANK CHARGES
208690,565735,BANK CHARGES,Bank Charges,1,2011-09-06 12:25:00,15.000,16904,United Kingdom,,,BANK CHARGES,BANK CHARGES
233494,568375,BANK CHARGES,Bank Charges,1,2011-09-26 17:01:00,15.000,13405,United Kingdom,,,BANK CHARGES,BANK CHARGES
233495,568375,BANK CHARGES,Bank Charges,1,2011-09-26 17:01:00,0.001,13405,United Kingdom,,,BANK CHARGES,BANK CHARGES
267272,571900,BANK CHARGES,Bank Charges,1,2011-10-19 14:26:00,15.000,13263,United Kingdom,,,BANK CHARGES,BANK CHARGES
292581,574546,BANK CHARGES,Bank Charges,1,2011-11-04 14:59:00,15.000,13651,United Kingdom,,,BANK CHARGES,BANK CHARGES
356444,581127,BANK CHARGES,Bank Charges,1,2011-12-07 12:45:00,15.000,16271,United Kingdom,,,BANK CHARGES,BANK CHARGES


Описание везде имеет значение "Bank Charges" - "Банковские сборы".\
Также удалим эти транзации.

In [48]:
data.drop(data[mask].index, inplace=True)

### Подушки (StockCode == PADS)

In [49]:
mask = data[FT.STOCK_CODE_PREFIX.value] == 'PADS'
data[mask]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNoPrefix,InvoiceNoPostfix,StockCodePrefix,StockCodePostfix
96504,550193,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-04-15 09:27:00,0.001,13952,United Kingdom,,,PADS,PADS
174753,561226,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-07-26 10:13:00,0.001,15618,United Kingdom,,,PADS,PADS
230589,568158,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-09-25 12:22:00,0.000,16133,United Kingdom,,,PADS,PADS
231748,568200,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-09-25 14:58:00,0.001,16198,United Kingdom,,,PADS,PADS


Описание везде "PADS TO MATCH ALL CUSHIONS" - "ПОДУШКИ, ПОДХОДЯЩИЕ КО ВСЕМ ПОДУШКАМ".\
Пользователи разные, цена практически нулевая.\
Оставим эти транзакции. \
Возможно, это какой-то приз или подарок, что все-же говорит об активности пользователя.

Посмотрим, какие значения префиксов и окончаний для `StockCode` у нас остались.

In [50]:
print('Уникальные префиксы StockCode: ')
print(data[FT.STOCK_CODE_PREFIX.value].unique())
print()
print('Уникальные окончания StockCode: ')
print(data[FT.STOCK_CODE_POSTFIX.value].unique())

Уникальные префиксы StockCode: 
['' 'C' 'PADS']

Уникальные окончания StockCode: 
['A' '' 'B' 'G' 'E' 'L' 'C' 'S' 'BL' 'N' 'D' 'F' 'T' 'H' 'M' 'R' 'K' 'P'
 'W' 'J' 'U' 'V' 'PADS' 'I' 'Y']


### Перевозка (StockCode == C2)

In [51]:
mask = data[FT.STOCK_CODE_PREFIX.value] == 'C'
data[mask].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNoPrefix,InvoiceNoPostfix,StockCodePrefix,StockCodePostfix
1276,536540,C2,CARRIAGE,1,2010-12-01 14:05:00,50.0,14911,EIRE,,,C,
7755,537368,C2,CARRIAGE,1,2010-12-06 12:40:00,50.0,14911,EIRE,,,C,
8025,537378,C2,CARRIAGE,1,2010-12-06 13:06:00,50.0,14911,EIRE,,,C,
11924,538002,C2,CARRIAGE,1,2010-12-09 11:48:00,50.0,14932,Channel Islands,,,C,
21223,539421,C2,CARRIAGE,1,2010-12-17 14:21:00,50.0,14016,EIRE,,,C,


In [52]:
data[mask][F.DESCRIPTION.value].value_counts()

CARRIAGE    133
Name: Description, dtype: int64

Все описания имеют значение "CARRIAGE" - "Перевозка".\
Удалим эти транзакции.

In [53]:
data.drop(data[mask].index, inplace=True)

Исследование показывает, что остальные окончания у поля `StockCode` принадлежат идентификаторам товаров.

Удалим поля `StockCodePrefix`, `StockCodePostfix` и `InvoiceNoPostfix`.\
Они нам больше не понадобятся.

In [54]:
columns_to_drop = [
    FT.STOCK_CODE_PREFIX.value, 
    FT.STOCK_CODE_POSTFIX.value, 
    FT.INVOICE_NO_POSTFIX.value,
]
data.drop(columns=columns_to_drop, inplace=True)

## Исследование транзакций с отрицательным количеством товаров  <a id="research_neq_quantity"></a>

Посмотрим подробнее на транзакции с отрицательным количеством товара.

In [55]:
mask = data[F.QUANTITY.value] < 0

# Выделим строки с отрицательным количеством товара в отдельную таблицу для дальнейших исследований
negative_quantity_data = data[mask]

negative_quantity_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNoPrefix
152,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom,C
843,C536506,22960,JAM MAKING SET WITH JARS,-6,2010-12-01 12:38:00,4.25,17897,United Kingdom,C
1294,C536543,22632,HAND WARMER RED RETROSPOT,-1,2010-12-01 14:30:00,2.10,17841,United Kingdom,C
1295,C536543,22355,CHARLOTTE BAG SUKI DESIGN,-2,2010-12-01 14:30:00,0.85,17841,United Kingdom,C
1296,C536548,22244,3 HOOK HANGER MAGIC GARDEN,-4,2010-12-01 14:33:00,1.95,12472,Germany,C


In [56]:
print('Количество транзакций с отрицательным количеством товаров: ', negative_quantity_data.shape[0])

Количество транзакций с отрицательным количеством товаров:  7524


Как видим, для таких транзаций значение столбца с номером счета-фактуры `InvoiceNo` начинается с `C`,\
что указывает на возврат товара.

Проверим, для всех ли строк это так.

In [57]:
# Проверка, что первой буквой в значении столбца InvoiceNo является "C"
mask = negative_quantity_data[FT.INVOICE_NO_PREFIX.value] == 'C'

print(
    'Количество строк, которых InvoiceNo начинается с "C": ', 
    negative_quantity_data[mask].shape[0]
)

Количество строк, которых InvoiceNo начинается с "C":  7524


Количество строк совпадает с общим количеством транзакций с отрицательным количеством товара.\
Поэтому можно сделать вывод, что это общее правило.

Проверим, что для транзакций с возвратом существуют транзации с покупкой.

У транзакций с покупкой должны совпадать:
- `StockCode` - код товара
- `CustomerID` - идентификатор клиента

Посмотрим, что происходит при совпадении этих данных у одного клиента.\
Для примера возьмем данные из первой строки.

In [58]:
mask = (data[F.CUSTOMER_ID.value] == '15311') & (data[F.STOCK_CODE.value] == '35004C')
data[mask]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNoPrefix
152,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom,C
6272,537195,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2010-12-05 13:55:00,4.65,15311,United Kingdom,
10788,C537805,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-08 13:18:00,4.65,15311,United Kingdom,C
16517,538651,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2010-12-13 15:07:00,4.65,15311,United Kingdom,
22467,C539640,35004C,SET OF 3 COLOURED FLYING DUCKS,-3,2010-12-20 15:27:00,4.65,15311,United Kingdom,C
24352,540157,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2011-01-05 11:41:00,4.65,15311,United Kingdom,
32915,541293,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2011-01-17 13:39:00,4.65,15311,United Kingdom,
43185,C542866,35004C,SET OF 3 COLOURED FLYING DUCKS,-2,2011-02-01 12:14:00,4.65,15311,United Kingdom,C


Видим, что клиент периодически покупает по 12 единиц товара.\
И через несколько дней делает возврат на меньшее количество единиц, чем изначально купил.\
При этом первой найденной транзацией является возврат.\
То есть, можно предположить, что перед этим была совершена покупка, но данные о ней в таблицу не попали.

> Можно поступить следующим образом.\
Создать в таблице столбец `QuantityCanceled` и занести в него количество отмененных товаров, \
которые относятся к предыдущей покупке.\
Затем удалить из таблицы данные о возвратах.

Создадим функцию `get_quantity_canceled`, которая будет возвращать столбец `Series` \
с количеством возвращенных для каждой транзакции товаров.

Функция вынесена в файл [./helpers/prepare_data.py](./helpers/prepare_data.py)

In [59]:
data['QuantityCanceled'] = get_quantity_canceled(data)

Посмотрим на данные того же покупателя.

In [60]:
mask = (data[F.CUSTOMER_ID.value] == '15311') & (data[F.STOCK_CODE.value] == '35004C')
data[mask]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNoPrefix,QuantityCanceled
152,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom,C,0.0
6272,537195,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2010-12-05 13:55:00,4.65,15311,United Kingdom,,1.0
10788,C537805,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-08 13:18:00,4.65,15311,United Kingdom,C,0.0
16517,538651,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2010-12-13 15:07:00,4.65,15311,United Kingdom,,3.0
22467,C539640,35004C,SET OF 3 COLOURED FLYING DUCKS,-3,2010-12-20 15:27:00,4.65,15311,United Kingdom,C,0.0
24352,540157,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2011-01-05 11:41:00,4.65,15311,United Kingdom,,0.0
32915,541293,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2011-01-17 13:39:00,4.65,15311,United Kingdom,,2.0
43185,C542866,35004C,SET OF 3 COLOURED FLYING DUCKS,-2,2011-02-01 12:14:00,4.65,15311,United Kingdom,C,0.0


Видим, что в столбце `QuantityCanceled` корректно отображается количество возвращенных товаров.

Теперь удалим строки с отрицательным количеством товаров.

In [61]:
mask = data[F.QUANTITY.value] < 0
data.drop(data[mask].index, inplace=True)

И также удалим столбец `InvoiceNoPrefix`, потому что он больше не содержит уникальных значений.

In [62]:
data.drop(columns=[FT.INVOICE_NO_PREFIX.value], inplace=True)

Посмотрим на общий процент возвращенных товаров.

In [63]:
return_percentage = round(data[F.QUANTITY_CANCELED.value].sum() * 100 / data[F.QUANTITY.value].sum(), 2)

print(f'Из общего количества купленных товаров {return_percentage}% вернули.')

Из общего количества купленных товаров 3.43% вернули.


Посмотрим, сколько процентов заказов прошло с возвратами.

In [64]:
orders_canceled_data = data.groupby(by=[F.CUSTOMER_ID.value, F.INVOICE_NO.value]).agg(
    QuantityCanceledSum=(F.QUANTITY_CANCELED.value, 'sum'),
)
orders_canceled_data['isOrderWithCanceled'] = orders_canceled_data['QuantityCanceledSum'] > 0
orders_canceled_data['isOrderWithCanceled'].value_counts(normalize=True)

False    0.832011
True     0.167989
Name: isOrderWithCanceled, dtype: float64

С возвратом прошло около 17% заказов.

## Исследование транзакций с нулевой стоимостью товаров  <a id="research_zero_price"></a>

Посмотрим на характеристики распределения числовых признаков.

In [65]:
data.describe()

,Quantity,UnitPrice,QuantityCanceled
count,351327.000000,351327.000000,351327.000000
mean,13.144079,2.860003,0.450620
std,144.850461,4.159425,136.978729
min,1.000000,0.000000,0.000000
25%,2.000000,1.250000,0.000000
50%,6.000000,1.950000,0.000000
75%,12.000000,3.750000,0.000000
max,80995.000000,649.500000,80995.000000


Посмотрим на товары с нулевой ценой

In [66]:
mask = data[F.UNIT_PRICE.value] == 0
data[mask].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityCanceled
6364,537197,22841,ROUND CAKE TIN VINTAGE GREEN,1,2010-12-05 14:02:00,0.0,12647,Germany,0.0
20276,539263,22580,ADVENT CALENDAR GINGHAM SACK,4,2010-12-16 14:36:00,0.0,16560,United Kingdom,0.0
22862,539722,22423,REGENCY CAKESTAND 3 TIER,10,2010-12-21 13:45:00,0.0,14911,EIRE,0.0
26150,540372,22090,PAPER BUNTING RETROSPOT,24,2011-01-06 16:41:00,0.0,13081,United Kingdom,0.0
26152,540372,22553,PLASTERS IN TIN SKULLS,24,2011-01-06 16:41:00,0.0,13081,United Kingdom,0.0


In [67]:
print('Количество товаров с нулевой ценой: ', data[mask].shape[0])

Количество товаров с нулевой ценой:  33


Похоже, что это подарки, которые добавляются к заказу при выполнении пользователями каких-то условий.\
Оставим их.

## Добавление признака TotalPrice (общая стоимость покупки)  <a id="total_price"></a>

Добавим в датасет поле `TotalPrice` с общей ценой покупки.\
Рассчитаем ее как:

$$ общая \ цена = цена \ за \ единицу \ товара \times (количество \ товаров \ в \ покупке - количество \ возвращённых \ товаров) $$

In [68]:
data[F.TOTAL_PRICE.value] = data.apply(
    lambda row: get_total_price(
        row[F.UNIT_PRICE.value], 
        row[F.QUANTITY.value], 
        row[F.QUANTITY_CANCELED.value],
    ), axis=1
)

Посмотрим на характеристики распределения стоимости покупок.

In [69]:
data[F.TOTAL_PRICE.value].describe()

count    351327.000000
mean         21.265839
std          69.434418
min           0.000000
25%           4.680000
50%          11.700000
75%          19.620000
max        7144.720000
Name: TotalPrice, dtype: float64

> Код выполненных преобразований продублирован в функции `get_prepared_data` \
в файле [./helpers/prepare_data.py](./helpers/prepare_data.py)